<a href="https://colab.research.google.com/github/saksham751/Neural-Network-from-Scratch/blob/main/neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.mnist import load_data
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [153]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward_propagation(self, input):
        raise NotImplementedError
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [154]:
class complateLayer(Layer):
  
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        #print(output_error.shape, self.weights.T.shape)
        #print(self.input.T.shape, output_error.shape)
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T.reshape(self.input.T.shape[0],1), output_error)
        
        self.weights -= learning_rate * weights_error
        self.bias = self.bias-learning_rate * output_error
        return input_error

In [155]:

class activationFunction(Layer):
    def __init__(self, activation, activationDer):
        self.activation = activation
        self.activationDer = activationDer

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activationDer(self.input) * output_error

In [156]:
    def sig(x):
      return 1/(1 + np.exp(-x))

    def sig_der(x):
      return sig(x)*(1-sig(x))
  
    def tanh(x):
      return np.tanh(x)

    def tanh_prime(x):
      return 1-np.tanh(x)**2

    def relu(x):
      return np.maximum(x, 0)

    def relu_der(x):
      return 1. * (x > 0)

    def linear(x):
      return x

    def linear_der(x):
      return 1

    

In [157]:

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mseDer(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [158]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):

        samples = len(input_data)
        result = []
        for i in range(samples):

            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate):

        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
  
                output = x_train[j]
                
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)

                error = self.loss_prime(y_train[j], output)
                #error.reshape(784,1)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
                #print('error=%f'%(err/j))

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [159]:
def classAccuracy( y_test,res):
  cor=0
  for x,y in zip(y_test,res):
    cor+=x==y
  acc=cor/len(res)*100
  return acc

In [160]:
def mseRegression(y_test,res):
  errors = mean_squared_error(y_test, res)
  return errors

In [161]:
#(x_train, y_train), (x_test, y_test)= load_data( path="mnist.npz" )
iris = datasets.load_iris()
x, y = iris.data, iris.target

In [162]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [163]:
x_train.shape[1]
classes=np.unique(y_train).size
y_train

array([2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 1, 0, 0, 2, 0,
       0, 1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1, 2, 0,
       2, 1, 1, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1,
       0, 0, 1, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0,
       0, 1, 2, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2,
       1, 1, 2, 2, 1, 0, 1, 1, 2, 2])

In [164]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_train.shape

(120, 3)

In [165]:
network1 = Network()
x1=x_train.shape[1]
x2=4
network1.add(complateLayer(x1, x2))
network1.add(activationFunction(tanh, tanh_prime))
network1.add(complateLayer(x2, classes))
network1.add(activationFunction(tanh, tanh_prime))


In [166]:
# x_train.reshape(x_train.shape[0],1,784)
# x_new2=x_train.reshape(x_train.shape[0],1,784)
# x_new=np.delete(x_new2,np.s_[0:50000],axis=0)
# x_new.shape

In [167]:
# y_new2=y_train.reshape(y_train.shape[0],1)
# y_new=np.delete(y_new2,np.s_[0:50000],axis=0)
# y_new.shape


In [168]:
x_train.shape

(120, 4)

In [169]:
network1.use(mse, mseDer)

network1.fit(x_train ,y_train, epochs=100, learning_rate=0.001)

epoch 1/100   error=0.586298
epoch 2/100   error=0.579855
epoch 3/100   error=0.572782
epoch 4/100   error=0.565164
epoch 5/100   error=0.557183
epoch 6/100   error=0.549123
epoch 7/100   error=0.541354
epoch 8/100   error=0.534261
epoch 9/100   error=0.528157
epoch 10/100   error=0.523198
epoch 11/100   error=0.519364
epoch 12/100   error=0.516500
epoch 13/100   error=0.514385
epoch 14/100   error=0.512800
epoch 15/100   error=0.511560
epoch 16/100   error=0.510525
epoch 17/100   error=0.509600
epoch 18/100   error=0.508722
epoch 19/100   error=0.507847
epoch 20/100   error=0.506948
epoch 21/100   error=0.506006
epoch 22/100   error=0.505006
epoch 23/100   error=0.503935
epoch 24/100   error=0.502780
epoch 25/100   error=0.501530
epoch 26/100   error=0.500171
epoch 27/100   error=0.498687
epoch 28/100   error=0.497059
epoch 29/100   error=0.495267
epoch 30/100   error=0.493286
epoch 31/100   error=0.491084
epoch 32/100   error=0.488627
epoch 33/100   error=0.485871
epoch 34/100   erro

In [171]:
x_test

array([[6.3, 2.5, 4.9, 1.5],
       [6.8, 3. , 5.5, 2.1],
       [6.4, 2.8, 5.6, 2.2],
       [5.6, 3. , 4.1, 1.3],
       [4.9, 3.6, 1.4, 0.1],
       [6. , 3. , 4.8, 1.8],
       [6.3, 2.3, 4.4, 1.3],
       [4.4, 3.2, 1.3, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [5.5, 2.6, 4.4, 1.2],
       [6.9, 3.1, 5.1, 2.3],
       [5.5, 4.2, 1.4, 0.2],
       [5.2, 2.7, 3.9, 1.4],
       [6.5, 3. , 5.5, 1.8],
       [7.7, 3. , 6.1, 2.3],
       [6.5, 3. , 5.8, 2.2],
       [5.5, 3.5, 1.3, 0.2],
       [4.3, 3. , 1.1, 0.1],
       [6.1, 2.9, 4.7, 1.4],
       [4.8, 3. , 1.4, 0.3],
       [5.2, 3.4, 1.4, 0.2],
       [6.3, 2.8, 5.1, 1.5],
       [4.8, 3.4, 1.9, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [5.1, 3.8, 1.6, 0.2],
       [5.4, 3.4, 1.7, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.6, 2.8, 4.9, 2. ],
       [7.7, 3.8, 6.7, 2.2],
       [5. , 3.6, 1.4, 0.2]])

In [172]:
out = network1.predict(x_test)
out=[x[0] for x in out]

In [173]:
out

[array([0.30097139, 0.39599563, 0.35500636]),
 array([0.30101626, 0.39630336, 0.35526238]),
 array([0.30102105, 0.39633621, 0.35528972]),
 array([0.30083515, 0.39506095, 0.35422884]),
 array([0.28003292, 0.24466375, 0.2311292 ]),
 array([0.30098316, 0.39607638, 0.35507354]),
 array([0.30087978, 0.3953672 , 0.35448357]),
 array([0.28126372, 0.25395557, 0.23864264]),
 array([0.28352547, 0.27089297, 0.25236315]),
 array([0.30089562, 0.39547593, 0.35457402]),
 array([0.3010076 , 0.39624396, 0.35521296]),
 array([0.27905067, 0.23723012, 0.22512462]),
 array([0.30081505, 0.39492308, 0.35411417]),
 array([0.30101248, 0.39627741, 0.35524079]),
 array([0.30102567, 0.39636788, 0.35531607]),
 array([0.30102383, 0.39635523, 0.35530555]),
 array([0.27627839, 0.21615602, 0.20812958]),
 array([0.2746071 , 0.20343435, 0.19788767]),
 array([0.30094721, 0.3958298 , 0.3548684 ]),
 array([0.28342164, 0.27009569, 0.25171686]),
 array([0.28032644, 0.24687484, 0.23291646]),
 array([0.30098693, 0.3961022 , 0.

In [178]:
res=np.argmax(out,axis=1)
res

array([1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0])

In [179]:
y_test

array([1, 2, 2, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2,
       0, 2, 0, 0, 0, 2, 2, 0])

In [180]:
classAccuracy(y_test,res)

56.666666666666664

In [100]:
x_train.shape

(120, 4)

In [101]:
db = datasets.load_diabetes()
x, y = db.data, db.target

In [102]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)

In [103]:
x_train
y_train

array([170.,  77.,  49.,  66., 144., 113., 233., 162., 121.,  88., 302.,
       128.,  52., 178.,  90., 164., 135., 103., 200., 178., 261.,  64.,
        59.,  79.,  47., 107.,  39., 151., 104., 217., 232.,  55., 245.,
       131.,  64., 222., 249., 128., 293., 138., 273., 158., 168., 103.,
       140., 146., 277., 116., 233.,  94., 295., 191., 241., 154., 141.,
       252., 198., 288., 101., 179., 129., 174.,  91.,  84., 177., 219.,
        67., 113.,  70.,  53., 150.,  40., 258., 168., 283., 252.,  43.,
        74.,  78., 209., 270.,  90.,  73., 310., 102., 115.,  85., 104.,
        68., 101., 139.,  63.,  72.,  58., 131.,  71.,  98.,  88., 277.,
       104., 167., 108., 111., 214., 135., 196., 189.,  77., 225.,  60.,
        67., 292., 143., 281.,  25., 200.,  91., 263.,  95., 144., 232.,
       109.,  51.,  64., 283., 259., 214.,  60., 111., 258.,  68., 174.,
       118., 113., 306.,  70.,  78., 191., 220., 183., 124.,  44.,  63.,
       114., 229.,  77., 168., 262.,  91.,  85., 12

In [104]:
# from keras.utils import np_utils
# y_train = np_utils.to_categorical(y_train)
# y_train.shape

In [105]:
y_train

array([170.,  77.,  49.,  66., 144., 113., 233., 162., 121.,  88., 302.,
       128.,  52., 178.,  90., 164., 135., 103., 200., 178., 261.,  64.,
        59.,  79.,  47., 107.,  39., 151., 104., 217., 232.,  55., 245.,
       131.,  64., 222., 249., 128., 293., 138., 273., 158., 168., 103.,
       140., 146., 277., 116., 233.,  94., 295., 191., 241., 154., 141.,
       252., 198., 288., 101., 179., 129., 174.,  91.,  84., 177., 219.,
        67., 113.,  70.,  53., 150.,  40., 258., 168., 283., 252.,  43.,
        74.,  78., 209., 270.,  90.,  73., 310., 102., 115.,  85., 104.,
        68., 101., 139.,  63.,  72.,  58., 131.,  71.,  98.,  88., 277.,
       104., 167., 108., 111., 214., 135., 196., 189.,  77., 225.,  60.,
        67., 292., 143., 281.,  25., 200.,  91., 263.,  95., 144., 232.,
       109.,  51.,  64., 283., 259., 214.,  60., 111., 258.,  68., 174.,
       118., 113., 306.,  70.,  78., 191., 220., 183., 124.,  44.,  63.,
       114., 229.,  77., 168., 262.,  91.,  85., 12

In [106]:
network1 = Network()
x1=x_train.shape[1]
x2=x1*2
network1.add(complateLayer(x1, x2))
network1.add(activationFunction(relu, relu_der))
network1.add(complateLayer(x2, 1))
network1.add(activationFunction(linear, linear_der))


In [107]:
network1.use(mse, mseDer)

network1.fit(x_train ,y_train, epochs=50, learning_rate=0.001)

epoch 1/50   error=6578.015440
epoch 2/50   error=5277.579121
epoch 3/50   error=4899.264598
epoch 4/50   error=4711.458328
epoch 5/50   error=4596.292207
epoch 6/50   error=4511.709502
epoch 7/50   error=4449.469504
epoch 8/50   error=4396.270297
epoch 9/50   error=4350.789106
epoch 10/50   error=4289.574845
epoch 11/50   error=4247.437310
epoch 12/50   error=4240.904832
epoch 13/50   error=4180.151938
epoch 14/50   error=4161.253471
epoch 15/50   error=4134.557650
epoch 16/50   error=4087.840996
epoch 17/50   error=4017.846022
epoch 18/50   error=3993.240485
epoch 19/50   error=3942.590554
epoch 20/50   error=3892.723513
epoch 21/50   error=3869.341538
epoch 22/50   error=3828.308232
epoch 23/50   error=3793.301857
epoch 24/50   error=3762.493732
epoch 25/50   error=3728.629726
epoch 26/50   error=3703.562468
epoch 27/50   error=3677.821243
epoch 28/50   error=3667.123457
epoch 29/50   error=3646.895555
epoch 30/50   error=3624.088102
epoch 31/50   error=3599.375500
epoch 32/50   err

In [108]:
out = network1.predict(x_test)
print(out)

[array([[115.65783766]]), array([[111.376643]]), array([[166.16684308]]), array([[113.15381731]]), array([[118.04168146]]), array([[116.18441223]]), array([[197.36611222]]), array([[107.927114]]), array([[107.19869863]]), array([[106.28235947]]), array([[107.27219309]]), array([[131.13632358]]), array([[120.13300194]]), array([[149.77485926]]), array([[168.23321298]]), array([[126.47864508]]), array([[106.58006787]]), array([[108.16013386]]), array([[132.18837785]]), array([[146.62211493]]), array([[117.16770659]]), array([[178.14831313]]), array([[118.31010371]]), array([[106.28235947]]), array([[107.22788794]]), array([[139.80894663]]), array([[106.65898378]]), array([[106.28235947]]), array([[114.70926271]]), array([[125.62311839]]), array([[106.41657311]]), array([[178.78021484]]), array([[138.88826607]]), array([[160.23891048]]), array([[141.5258807]]), array([[106.28235947]]), array([[106.28235947]]), array([[106.28235947]]), array([[167.21407607]]), array([[106.28235947]]), arra

In [109]:
y_test

array([185., 150., 246., 184., 110., 202., 336.,  69.,  69.,  87.,  66.,
       164., 265., 198., 248., 171., 102., 182., 262., 233., 151., 275.,
       230.,  39.,  42., 122.,  81.,  31., 156., 161.,  80., 215., 310.,
       163., 265.,  84.,  54., 137., 248.,  72., 248.,  59.,  97.,  61.,
        72., 296.,  55.,  75.,  53., 173., 158.,  96., 140., 186.,  93.,
       123., 237., 138., 292.,  78., 182., 127., 321., 109., 212.,  45.,
        96., 109.,  89.,  96.,  60.,  51., 200., 265.,  68.,  91., 172.,
       115., 125.,  53.,  63., 259., 214., 195., 258., 235., 173.,  59.,
       219.])